In [1]:
# read the data 
import numpy as np
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = 10, 10
from utilities import *
import math
from sklearn.model_selection import train_test_split
import pandas as pd
from itertools import combinations

In [2]:
#sc = pyspark.SparkContext()

In [3]:
def process_inhval(*args):
    for val in args: 
        if val == 'NA':
            yield 0
        else:
            yield float(val)

In [4]:
readfilename = 'PI_DataSet_6_19_all.txt'
r = open(readfilename)
header = process_line(next(r))
inh_start = header.index('FPV') 
inh_end = header.index('DRV')+1
vec_start = header.index('count_vec_start')
vec_end = len(header)
vals = []
for line in r:
    if line!= '\n':
        line = process_line(line)
        token = list(process_inhval(*line[inh_start:inh_end])) + [int(x) for x in line[vec_start:vec_end]]
        vals.append(token)
r.close()

In [5]:
vals = np.array(vals)

In [6]:
ind_inh = {v:i for i,v in enumerate(header[inh_start:inh_end])}

In [7]:
def filter_inhs(data, ind_dict, *args):
    vec_length = 210
    names = sorted([(ind_dict[name],name) for i,name in enumerate(args)])
    inds = list(map(lambda x:x[0], names))
    names = list(map (lambda x:x[1], names))
    tmp = np.hstack((data[:,inds],data[:,8:]))
    df = pd.DataFrame(tmp, columns = ['c'+str(i) for i in range(len(tmp[0]))])
    query_string = ' & '.join(['c'+str(i)+">0" for i in range(len(inds))])
    df = df.query(query_string)
    for i,name in enumerate(names):
        df[names[i]] = np.where(df['c'+str(i)] > 3.0 , '1', '0')
    df['target'] = df[names].apply(lambda x:int(''.join(x),2),axis = 1)
    return df.iloc[:,len(names):(-len(names)-1)].values,df['target'].values.reshape(-1,1)

In [8]:
X,y = filter_inhs(vals,ind_inh,'DRV','TPV','IDV')

/home/dshah8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/dshah8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [9]:
max_ = X.max()
X = X/max_

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [11]:
X_train.shape, y_train.shape

((136593, 210), (136593, 1))

In [12]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import time 

In [13]:
#model = AdaBoostClassifier( DecisionTreeClassifier(max_depth=2), n_estimators=400,learning_rate=1.5,algorithm="SAMME")

In [28]:
lr = 0.5
model = AdaBoostClassifier( DecisionTreeClassifier(max_depth=2), n_estimators=100,learning_rate=lr,algorithm="SAMME")
t0 = time.time()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_pred,y_test)
print ("accuracy:", acc,"time:",time.time()-t0)
print(precision_recall_fscore_support(y_test,y_pred,average='weighted'))

/home/dshah8/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy: 0.9857903029222034 time: 109.30507349967957
(0.9855849590015399, 0.9857903029222034, 0.9849079486918845, None)


/home/dshah8/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
